In [152]:
import numpy as np
from scipy.special import binom

In [153]:
q1 = 2
pos1 = 0.11+0.22j
q2 = -1.5
pos2 = 0.93+0.81j

p = 4

In [154]:
# gen multipole
l0_a = [q1]
for k in range(1,p):
    l0_a.append(-q1 * (pos1-(0.125+0.125j))**k / k)

l15_a = [q2]
for k in range(1,p):
    l15_a.append(-q2 * (pos2-(0.875+0.875j))**k / k)

l0_a, l15_a

([2, (0.03-0.19j), (0.0088+0.00285j), (-0.0002685+0.0005288333333333333j)],
 [-1.5,
  (0.08250000000000007-0.09749999999999992j),
  (-0.0008999999999999903-0.0053625j),
  (-0.0002653749999999995-0.00015762500000000065j)])

In [155]:
# M2M to level 1
m0_b = [l0_a[0]]
for l in range(1,p):
    bl = -l0_a[0] * ((0.125+0.125j) - (0.25+0.25j))**l / l
    for k in range(1,l+1):
        bl += l0_a[k] * ((0.125+0.125j) - (0.25+0.25j))**(l-k) * binom(l-1,k-1)
    m0_b.append(bl)

m3_b = [l15_a[0]]
for l in range(1,p):
    bl = -l15_a[0] * ((0.875+0.875j) - (0.75+0.75j))**l / l
    for k in range(1,l+1):
        bl += l15_a[k] * ((0.875+0.875j) - (0.75+0.75j))**(l-k) * binom(l-1,k-1)
    m3_b.append(bl)

m0_b, m3_b

([2,
  (0.28+0.06j),
  (-0.0187-0.0084j),
  (0.0015773333333333334+0.0011579999999999997j)],
 [-1.5,
  (0.2700000000000001+0.09000000000000008j),
  (0.021600000000000008+0.01620000000000002j),
  (0.0019440000000000004+0.002808000000000004j)])

In [156]:
# M2M to level 0
n_b = [m0_b[0] + m3_b[0]]
for l in range(1,p):
    bl = -m0_b[0] * ((0.25+0.25j) - (0.5+0.5j))**l / l
    bl -= m3_b[0] * ((0.75+0.75j) - (0.5+0.5j))**l / l
    for k in range(1,l+1):
        bl += m0_b[k] * ((0.25+0.25j) - (0.5+0.5j))**(l-k) * binom(l-1,k-1)
        bl += m3_b[k] * ((0.75+0.75j) - (0.5+0.5j))**(l-k) * binom(l-1,k-1)
    n_b.append(bl)

n_b

[0.5,
 (1.425+1.0250000000000001j),
 (-0.007100000000000002-0.018449999999999946j),
 (-0.04383700000000001+0.14162433333333338j)]

In [157]:
# M2L for interaction list

z0 = (0.875+0.875j) - (0.125+0.125j)
l0_c = [l15_a[0] * np.log(-z0)]
for k in range(1,p):
    l0_c[0] += (-1)**k * l15_a[k] / z0**k

for l in range(1,p):
    cl = -l15_a[0] / (l * z0**l)
    term = 0
    for k in range(1,p):
        term += ((-1)**k * l15_a[k] / z0**k) * binom(l+k-1, k-1)
    cl += (1/z0**l) * term
    l0_c.append(cl)


z0 = -z0
l15_c = [l0_a[0] * np.log(-z0)]
for k in range(1,p):
    l15_c[0] += (-1)**k * l0_a[k] / z0**k
    
for l in range(1,p):
    cl = -l0_a[0] / (l * z0**l)
    term = 0
    for k in range(1,p):
        term += ((-1)**k * l0_a[k] / z0**k) * binom(l+k-1, k-1)
    cl += (1/z0**l) * term
    l15_c.append(cl)

l0_c, l15_c

([(-0.08316779526080624+3.6548410686218507j),
  (1.0807487407407406-0.919618074074074j),
  (0.10746311111111106-0.6625039012345678j),
  (-0.2190231001371742-0.36206705075445805j)],
 [(0.014122196150210637+1.4161531663010694j),
  (1.5086376296296296-1.2916057283950617j),
  (-0.15205241152263374+0.9744281810699588j),
  (-0.34309735711019657-0.555836049382716j)])

In [158]:
# evaluate particle potential

pot1 = 0
for l in range(p):
    pot1 += l0_c[l] * (pos1 - (0.125+0.125j))**l

pot2 = 0
for l in range(p):
    pot2 += l15_c[l] * (pos2 - (0.875+0.875j))**l

pot1, pot2

((-0.015224542072767836+3.7768581521370224j),
 (0.020299389430357088+1.2473744376635567j))

In [159]:
dir1 = q2 * np.log(abs(pos1-pos2))
dir2 = q1 * np.log(abs(pos1-pos2))

dir1, dir2

(-0.015219527450822064, 0.02029270326776275)

In [160]:
pot1.real-dir1, pot2.real-dir2

(-5.014621945772027e-06, 6.686162594336104e-06)

In [165]:
def book_M2L(cell_centre, interactor_centre, interactor_expansion, p):
    cell_expansion = np.zeros(p, dtype=complex)

    z0 = interactor_centre - cell_centre

    cell_expansion[0] = interactor_expansion[0] * np.log(-z0)

    for k in range(1,p):
        cell_expansion[0] += (-1)**k * interactor_expansion[k] / z0**k

    for l in range(1,p):
        cl = -interactor_expansion[0] / (l * z0**l)
        term = 0
        for k in range(1,p):
            term += ((-1)**k * l15_a[k] / z0**k) * binom(l+k-1, k-1)
        cl += (1/z0**l) * term
        cell_expansion[l] = cl
    
    return cell_expansion

In [192]:

def cell_M2L(cell_centre, interactor_centre, interactor_expansion, p):
    cell_expansion = np.zeros(p, dtype=complex)

    z0 = interactor_centre - cell_centre # local expansion 'about origin' (so about cell)

    minus_and_plus = np.empty(p-1)
    minus_and_plus[::2] = -1
    minus_and_plus[1::2] = 1

    k_vals = np.arange(1, p)
    l_vals = np.arange(1, p)

    interactor_multipole = interactor_expansion

    minus_bk_over_z0k = minus_and_plus * interactor_multipole[1:] / z0**k_vals

    cell_expansion[0] += interactor_multipole[0] * np.log(-z0) + np.sum(minus_bk_over_z0k)

    cell_expansion[1:] += - interactor_multipole[0] / (l_vals * z0**l_vals) \
                    + (1/z0**l_vals) * np.sum(minus_bk_over_z0k * binom(l_vals[:,np.newaxis] + k_vals - 1, k_vals-1), axis=1)
    
    return cell_expansion

In [193]:

np.array(l0_c), \
book_M2L(0.125+0.125j, 0.875+0.875j, np.array(l15_a),p), \
cell_M2L(0.125+0.125j, 0.875+0.875j, np.array(l15_a),p)

(array([-0.0831678 +3.65484107j,  1.08074874-0.91961807j,
         0.10746311-0.6625039j , -0.2190231 -0.36206705j]),
 array([-0.0831678 +3.65484107j,  1.08074874-0.91961807j,
         0.10746311-0.6625039j , -0.2190231 -0.36206705j]),
 array([-0.0831678 +3.65484107j,  1.08074874-0.91961807j,
         0.10746311-0.6625039j , -0.2190231 -0.36206705j]))

In [191]:
minus_and_plus = np.empty(p-1)
minus_and_plus[::2] = -1
minus_and_plus[1::2] = 1

k_vals = np.arange(1, p)
l_vals = np.arange(1, p)

interactor_multipole = np.array(l15_a)
minus_bk_over_z0k = minus_and_plus * interactor_multipole[1:] / z0**k_vals

nump = np.sum(minus_bk_over_z0k * binom(l_vals[:,np.newaxis] + k_vals - 1, k_vals-1), axis=1)

l=1
term = 0
for k in range(1,p):
    term += ((-1)**k * l15_a[k] / z0**k) * binom(l+k-1, k-1)

term, nump

((-0.019341777777777677-0.11764800000000002j),
 array([-0.01934178-0.117648j  , -0.02391689-0.116096j  ,
        -0.02842815-0.11429333j]))